In [3]:
############################################################################################
# KALMAN FILTER PROJECT 
# CODE FROM 
# https://app.mybinder.org:80/2203287855/notebooks/08-Designing-Kalman-Filters.ipynb

from numpy.random import randn
import numpy as np
import copy
class PosSensor1(object):
    def __init__(self, pos=(0, 0), vel=(0, 0), noise_std=1.):
        self.vel = vel
        self.noise_std = noise_std
        self.pos = [pos[0], pos[1]]
        
    def read(self):
        self.pos[0] += self.vel[0]
        self.pos[1] += self.vel[1]
        
        return [self.pos[0] + randn() * self.noise_std,
                self.pos[1] + randn() * self.noise_std]

In [11]:
# state variables 
from filterpy.kalman import KalmanFilter
tracker = KalmanFilter(dim_x=4, dim_z=2)
dt = 1.   # delta T; time step 1 second
ndim = 2
tracker.F = np.array([[1, dt, 0,  0],
                      [0,  1, 0,  0],
                      [0,  0, 1, dt],
                      [0,  0, 0,  1]])
tracker.F

array([[ 1.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  1.],
       [ 0.,  0.,  0.,  1.]])

In [12]:
# design the state transition function
from scipy.linalg import block_diag
from filterpy.common import Q_discrete_white_noise

# model white noise, assumed its a discrete time Wieder process, constant for earch time period
q = Q_discrete_white_noise(dim=ndim, dt=dt, var=0.001)
tracker.Q = block_diag(q, q)
tracker.Q

array([[ 0.00025,  0.0005 ,  0.     ,  0.     ],
       [ 0.0005 ,  0.001  ,  0.     ,  0.     ],
       [ 0.     ,  0.     ,  0.00025,  0.0005 ],
       [ 0.     ,  0.     ,  0.0005 ,  0.001  ]])

In [13]:
# design process noise matrix
from scipy.linalg import block_diag
from filterpy.common import Q_discrete_white_noise

# assuming noise in x and y are independent
q = Q_discrete_white_noise(dim=ndim, dt=dt, var=0.001)
tracker.Q = block_diag(q, q)
tracker.Q

array([[ 0.00025,  0.0005 ,  0.     ,  0.     ],
       [ 0.0005 ,  0.001  ,  0.     ,  0.     ],
       [ 0.     ,  0.     ,  0.00025,  0.0005 ],
       [ 0.     ,  0.     ,  0.0005 ,  0.001  ]])

In [14]:
# design the control function
# The KalmanFilter class initializes B to zero under the assumption that there is no control input,
# so there is no code to write.
tracker.B

0

In [17]:
# design the measurement function
# H changes from state to measurement, so the conversion is 𝖿𝖾𝖾𝗍=𝗆𝖾𝗍𝖾𝗋𝗌/0.3048. This yields
tracker.H = np.array([[1/0.3048, 0, 0,        0],
                      [0,        0, 1/0.3048, 0]])
tracker.H

array([[ 3.2808399,  0.       ,  0.       ,  0.       ],
       [ 0.       ,  0.       ,  3.2808399,  0.       ]])

In [19]:
# design the measurement noise matrix
# for now setting the variance for x and y to be 5 meters2
tracker.R = np.array([[5, 0],
                      [0, 5]])
tracker.R

array([[5, 0],
       [0, 5]])

In [20]:
# initial conditions
tracker.x = np.array([[0, 0, 0, 0]]).T
tracker.P = np.eye(4) * 500

In [31]:
# implementing the filter
from filterpy.stats import plot_covariance_ellipse
import matplotlib.pyplot as plt

R_std = 0.35
Q_std = 0.04

def tracker1():
    tracker = KalmanFilter(dim_x=4, dim_z=2)
    dt = 1.0   # time step
    ndim = 2
    
    tracker.F = np.array([[1, dt, 0,  0],
                          [0,  1, 0,  0],
                          [0,  0, 1, dt],
                          [0,  0, 0,  1]])
    tracker.u = 0.
    tracker.H = np.array([[1/0.3048, 0, 0, 0],
                          [0, 0, 1/0.3048, 0]])

    tracker.R = np.eye(2) * R_std**2
    q = Q_discrete_white_noise(dim=ndim, dt=dt, var=Q_std**2)
    tracker.Q = block_diag(q, q)
    tracker.x = np.array([[0, 0, 0, 0]]).T
    tracker.P = np.eye(4) * 500.
    return tracker

# simulate robot movement
N = 30
sensor = PosSensor1((0, 0), (2, .2), noise_std=R_std)

zs = np.array([np.array([sensor.read()]).T for _ in range(N)])

# run filter
robot_tracker = tracker1()
mu, cov, _, _ = robot_tracker.batch_filter(zs)

for x, P in zip(mu, cov):
    # covariance of x and y
    cov = np.array([[P[0, 0], P[2, 0]], 
                    [P[0, 2], P[2, 2]]])
    mean = (x[0, 0], x[2, 0])
    plot_covariance_ellipse(mean, cov=cov, fc='g', std=3, alpha=0.5)
    
zs *= .3048 # convert to meters

In [33]:
print(robot_tracker.P)

[[ 0.00656162  0.00277676  0.          0.        ]
 [ 0.00277676  0.00298088  0.          0.        ]
 [ 0.          0.          0.00656162  0.00277676]
 [ 0.          0.          0.00277676  0.00298088]]


In [34]:
zs

array([[[  6.18384624e-01],
        [ -3.01332942e-03]],

       [[  1.32856893e+00],
        [  2.98555329e-03]],

       [[  1.88858973e+00],
        [  2.02238757e-01]],

       [[  2.51211819e+00],
        [  1.93941008e-01]],

       [[  3.38458463e+00],
        [  3.46758425e-01]],

       [[  3.63110500e+00],
        [  3.04395240e-01]],

       [[  4.29332572e+00],
        [  3.38994990e-01]],

       [[  4.90939376e+00],
        [  5.00314133e-01]],

       [[  5.38212894e+00],
        [  5.34519162e-01]],

       [[  6.14270587e+00],
        [  6.93610331e-01]],

       [[  6.76011394e+00],
        [  6.77475663e-01]],

       [[  7.27238623e+00],
        [  8.90686835e-01]],

       [[  7.81334393e+00],
        [  6.89589840e-01]],

       [[  8.43847856e+00],
        [  8.93987015e-01]],

       [[  9.03374301e+00],
        [  8.50315696e-01]],

       [[  9.76145191e+00],
        [  1.01833330e+00]],

       [[  1.04946500e+01],
        [  1.17327070e+00]],

       [[  1.0

In [35]:
robot_tracker

In [56]:
plt.plot(mu[:, 0], mu[:, 2])
plt.plot(zs[:, 0], zs[:, 1])
plt.show()